In [2]:
#Import matplotlib.pyplot as plt and seaborn as sns 
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
#Import pandas as pd
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import sklearn.utils

In [3]:
df1 = pd.read_csv("C:/Users/HP/Downloads/footprint-nfa-2019-edition/footprint-nfa-2019-edition/GFN Country Code Concordance Table.csv")
df1.head()

,GFN_name,country_code,ISO Alpha-2 Code,ISO Alpha-3 Code
0,Armenia,1,AM,ARM
1,Afghanistan,2,AF,AFG
2,Albania,3,AL,ALB
3,Algeria,4,DZ,DZA
4,American Samoa,5,AS,ASM


In [4]:
df2 = pd.read_csv("C:/Users/HP/Downloads/footprint-nfa-2019-edition/footprint-nfa-2019-edition/NFA 2019 public_data.csv", low_memory=False)
df2.head()

,country,year,country_code,record,crop_land,grazing_land,forest_land,fishing_ground,built_up_land,carbon,total,QScore
0,Armenia,1992,1,AreaPerCap,0.140292,0.199546,0.097188051,0.036888,0.029320,0.000000,5.032351e-01,3A
1,Armenia,1992,1,AreaTotHA,483000.000000,687000.000000,334600,127000.000000,100943.000800,0.000000,1.732543e+06,3A
2,Armenia,1992,1,BiocapPerCap,0.159804,0.135261,0.084003213,0.013742,0.033398,0.000000,4.262086e-01,3A
3,Armenia,1992,1,BiocapTotGHA,550176.242700,465677.972200,289207.1078,47311.551720,114982.279300,0.000000,1.467355e+06,3A
4,Armenia,1992,1,EFConsPerCap,0.387510,0.189462,1.26E-06,0.004165,0.033398,1.114093,1.728629e+00,3A


In [5]:
df_big = pd.merge(df1, df2, how = "outer")
df_big.shape

(72239, 15)

In [6]:
#df_big = pd.merge(df1, df2, how = "inner")
df_big.shape

(72186, 15)

In [7]:
df_big.head()

,GFN_name,country_code,ISO Alpha-2 Code,ISO Alpha-3 Code,country,year,record,crop_land,grazing_land,forest_land,fishing_ground,built_up_land,carbon,total,QScore
0,Armenia,1,AM,ARM,Armenia,1992,AreaPerCap,0.140292,0.199546,0.097188051,0.036888,0.029320,0.000000,5.032351e-01,3A
1,Armenia,1,AM,ARM,Armenia,1992,AreaTotHA,483000.000000,687000.000000,334600,127000.000000,100943.000800,0.000000,1.732543e+06,3A
2,Armenia,1,AM,ARM,Armenia,1992,BiocapPerCap,0.159804,0.135261,0.084003213,0.013742,0.033398,0.000000,4.262086e-01,3A
3,Armenia,1,AM,ARM,Armenia,1992,BiocapTotGHA,550176.242700,465677.972200,289207.1078,47311.551720,114982.279300,0.000000,1.467355e+06,3A
4,Armenia,1,AM,ARM,Armenia,1992,EFConsPerCap,0.387510,0.189462,1.26E-06,0.004165,0.033398,1.114093,1.728629e+00,3A


In [8]:
df_big.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72186 entries, 0 to 72185
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   GFN_name          72186 non-null  object 
 1   country_code      72186 non-null  int64  
 2   ISO Alpha-2 Code  70826 non-null  object 
 3   ISO Alpha-3 Code  71082 non-null  object 
 4   country           72186 non-null  object 
 5   year              72186 non-null  int64  
 6   record            72186 non-null  object 
 7   crop_land         51714 non-null  float64
 8   grazing_land      51714 non-null  float64
 9   forest_land       51714 non-null  object 
 10  fishing_ground    51713 non-null  float64
 11  built_up_land     51713 non-null  float64
 12  carbon            51713 non-null  float64
 13  total             72177 non-null  float64
 14  QScore            72185 non-null  object 
dtypes: float64(6), int64(2), object(7)
memory usage: 8.3+ MB


In [9]:
#check distribution of target variable
df_big['QScore'].value_counts()

QScore
3A    51481
2A    10576
2B    10096
1A       16
1B       16
Name: count, dtype: int64

In [10]:
df_big.isna().sum()

GFN_name                0
country_code            0
ISO Alpha-2 Code     1360
ISO Alpha-3 Code     1104
country                 0
year                    0
record                  0
crop_land           20472
grazing_land        20472
forest_land         20472
fishing_ground      20473
built_up_land       20473
carbon              20473
total                   9
QScore                  1
dtype: int64

In [11]:
#for simplicity, we will drop the rows with missing values.
df_big = df_big.dropna()
df_big.isna().sum()

GFN_name            0
country_code        0
ISO Alpha-2 Code    0
ISO Alpha-3 Code    0
country             0
year                0
record              0
crop_land           0
grazing_land        0
forest_land         0
fishing_ground      0
built_up_land       0
carbon              0
total               0
QScore              0
dtype: int64

In [12]:
#Recheck the distribution of target variable
df_big['QScore'].value_counts()

QScore
3A    50369
2A      224
1A       16
Name: count, dtype: int64

In [38]:
#An obvious change in our target variable after removing the missing values is that there are only three classes left #and from the distribution of the 3 classes, we can see that there is an obvious imbalance between the classes. #There are methods that can be applied to handle this imbalance such as oversampling and undersampling.
#Oversampling involves increasing the number of instances in the class with fewer instances while undersampling #involves reducing the data points in the class with more instances.
#For now, we will convert this to a binary classification problem by combining class '2A' and '1A'.
df_big['QScore'] = df_big['QScore'].replace(['1A'], '2A')
df_big.QScore.value_counts()

QScore
3A    50369
2A      240
Name: count, dtype: int64

In [40]:
df_big_2A = df_big[df_big.QScore=='2A']
df_big_3A = df_big[df_big.QScore=='3A'].sample(350)
data_df_big = pd.concat([df_big_2A, df_big_3A])
#data_df_big = df_big_2A.append(df_big_3A)

In [41]:
data_df_big = sklearn.utils.shuffle(data_df_big)
data_df_big = data_df_big.reset_index(drop=True)
print(data_df_big.shape)
data_df_big.QScore.value_counts()

(590, 15)


QScore
3A    350
2A    240
Name: count, dtype: int64

In [42]:
#more preprocessing
data_df_big = data_df_big.drop(columns=['country_code', 'country', 'year'])
x = data_df_big.drop(columns='QScore')
y = data_df_big['QScore']

In [43]:
y.shape

(590,)

In [44]:
x.shape

(590, 11)

In [45]:
#split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)
y_train.value_counts()


QScore
3A    249
2A    164
Name: count, dtype: int64

In [46]:
y_test.shape

(177,)

In [47]:
print(y_test)

225    3A
14     2A
85     3A
418    3A
132    2A
       ..
346    3A
369    3A
140    2A
533    3A
171    2A
Name: QScore, Length: 177, dtype: object


In [48]:
print(y_test)

225    3A
14     2A
85     3A
418    3A
132    2A
       ..
346    3A
369    3A
140    2A
533    3A
171    2A
Name: QScore, Length: 177, dtype: object


In [49]:
y_train_1 = y_train.values
y_train_1

array(['3A', '2A', '2A', '3A', '2A', '2A', '2A', '3A', '3A', '2A', '3A',
       '3A', '3A', '3A', '3A', '3A', '2A', '2A', '2A', '3A', '2A', '2A',
       '3A', '3A', '3A', '2A', '3A', '2A', '3A', '2A', '3A', '2A', '2A',
       '2A', '3A', '3A', '3A', '2A', '2A', '3A', '3A', '3A', '3A', '2A',
       '3A', '2A', '2A', '3A', '3A', '2A', '2A', '2A', '3A', '3A', '3A',
       '3A', '3A', '2A', '2A', '3A', '3A', '3A', '3A', '3A', '3A', '3A',
       '3A', '2A', '2A', '3A', '3A', '2A', '3A', '2A', '3A', '2A', '3A',
       '3A', '3A', '3A', '2A', '2A', '3A', '3A', '3A', '3A', '3A', '3A',
       '3A', '3A', '3A', '3A', '2A', '3A', '3A', '3A', '2A', '3A', '3A',
       '2A', '3A', '2A', '2A', '2A', '3A', '3A', '3A', '2A', '2A', '3A',
       '3A', '2A', '3A', '3A', '2A', '2A', '2A', '3A', '2A', '3A', '2A',
       '2A', '3A', '2A', '2A', '2A', '2A', '3A', '3A', '2A', '3A', '2A',
       '2A', '2A', '3A', '2A', '2A', '2A', '2A', '3A', '2A', '3A', '2A',
       '2A', '3A', '2A', '3A', '3A', '2A', '2A', '3

In [50]:
#encode categorical variable
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
x_train.record = encoder.fit_transform(x_train.record)
x_train.record

285    6
113    4
18     1
76     5
206    7
      ..
277    7
9      7
359    0
192    1
559    3
Name: record, Length: 413, dtype: int32

In [51]:
fitted = encoder.fit(x_test.record)
x_test.record = fitted.transform(x_test.record)
x_test.record

225    7
14     3
85     1
418    4
132    0
      ..
346    7
369    2
140    5
533    3
171    5
Name: record, Length: 177, dtype: int32

In [52]:
x_train_1 = x_train.values.flatten().astype(str)
x_train_1

array(['Albania', 'AL', 'ALB', ..., '5447.766068', '0.0', '3649370.045'],
      dtype='<U38')

In [54]:
x_train = encoder.fit_transform(x_train_1).reshape(413,11)
x_train.shape

(413, 11)

In [55]:
y_train = encoder.fit_transform(y_train_1).reshape(-1,1)
y_train.shape

(413, 1)

In [56]:
import imblearn
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=1)
x_train_balanced, y_train_balanced = smote.fit_resample(x_train, y_train)
x_train_balanced, y_train_balanced

(array([[2564, 2551, 2552, ...,  142,  842, 1005],
        [2907, 2959, 2960, ...,  140, 1469, 1742],
        [2819, 2789, 2790, ..., 1053,    1, 2011],
        ...,
        [2676, 2689, 2688, ...,  137,  940, 1037],
        [2760, 2752, 2752, ..., 2184, 1518, 2020],
        [2606, 2586, 2582, ...,  264,    1, 1759]], dtype=int64),
 array([1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0,
        1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0,
        1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1,
        1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1,
        1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0,
        0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1,
        0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1,
        0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1,


In [57]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
normalised_train_df = scaler.fit_transform(x_train_balanced)
normalised_train_df
normalised_train_df = pd.DataFrame(normalised_train_df, columns = data_df_big.columns.drop(['record']))
normalised_train_df
#normalised_train_df['record'] = x_train_balanced['record']

,GFN_name,ISO Alpha-2 Code,ISO Alpha-3 Code,crop_land,grazing_land,forest_land,fishing_ground,built_up_land,carbon,total,QScore
0,0.002481,0.009615,0.007229,0.955109,0.271902,0.196063,0.245972,0.027537,0.055403,0.332806,0.225466
1,0.853598,0.990385,0.990361,0.836255,0.243883,0.162598,0.222397,0.131786,0.054617,0.580926,0.596376
2,0.635236,0.581731,0.580723,0.398461,0.992502,0.601181,0.862475,0.846971,0.413360,0.000000,0.731756
3,0.736973,0.742788,0.734940,0.903378,0.651144,0.940945,0.779961,0.620771,0.767780,0.969133,0.374434
4,0.372208,0.394231,0.392771,1.000000,0.504736,0.636220,0.703340,0.551534,0.511591,0.525920,0.675390
...,...,...,...,...,...,...,...,...,...,...,...
493,0.441687,0.437500,0.438554,0.398461,0.864641,0.742913,0.925737,0.514949,0.466798,0.000000,0.278812
494,0.550868,0.538462,0.537349,0.903378,0.970403,0.554724,0.858546,0.940991,0.529666,0.921646,0.546049
495,0.280397,0.341346,0.334940,0.869602,0.218627,0.055906,0.242829,0.133360,0.053438,0.371587,0.241570
496,0.488834,0.492788,0.489157,0.940573,0.526440,0.888583,0.893124,0.762392,0.857760,0.600317,0.736286


In [58]:
normalised_train_df = normalised_train_df.drop(columns='QScore')
normalised_train_df.head()

,GFN_name,ISO Alpha-2 Code,ISO Alpha-3 Code,crop_land,grazing_land,forest_land,fishing_ground,built_up_land,carbon,total
0,0.002481,0.009615,0.007229,0.955109,0.271902,0.196063,0.245972,0.027537,0.055403,0.332806
1,0.853598,0.990385,0.990361,0.836255,0.243883,0.162598,0.222397,0.131786,0.054617,0.580926
2,0.635236,0.581731,0.580723,0.398461,0.992502,0.601181,0.862475,0.846971,0.413360,0.000000
3,0.736973,0.742788,0.734940,0.903378,0.651144,0.940945,0.779961,0.620771,0.767780,0.969133
4,0.372208,0.394231,0.392771,1.000000,0.504736,0.636220,0.703340,0.551534,0.511591,0.525920


In [59]:
print(x_test)

           GFN_name ISO Alpha-2 Code ISO Alpha-3 Code  record     crop_land  \
225           Japan               JP              JPN       7  1.625921e+07   
14          Morocco               MA              MAR       3  8.723089e+06   
85          Lebanon               LB              LBN       1  3.250000e+05   
418         Belgium               BE              BEL       4  1.128352e+00   
132      Tajikistan               TJ              TJK       0  9.879850e-02   
..              ...              ...              ...     ...           ...   
346  United Kingdom               GB              GBR       7  3.156208e+07   
369     Philippines               PH              PHL       2  3.344299e-01   
140      Mauritania               MR              MRT       5  1.481184e+06   
533          Mexico               MX              MEX       3  3.947719e+07   
171   French Guiana               GF              GUF       5  2.520173e+04   

     grazing_land  forest_land  fishing_ground  bui

In [60]:
x_test_1 = x_test.values.flatten().astype(str)
x_test_1

array(['Japan', 'JP', 'JPN', ..., '1403.916309', '465489.697',
       '655725.37'], dtype='<U38')

In [62]:
x_test = encoder.fit_transform(x_test_1).reshape(177,11)
x_test.shape

(177, 11)

In [63]:
fitted =  scaler.fit(x_test)
normalised_test_df = fitted.transform(x_test)
normalised_test_df = pd.DataFrame(normalised_test_df, columns = data_df_big.columns.drop(['record'])).reset_index(drop=True)
normalised_test_df
#normalised_test_df['record'] = x_test['record']

,GFN_name,ISO Alpha-2 Code,ISO Alpha-3 Code,crop_land,grazing_land,forest_land,fishing_ground,built_up_land,carbon,total,QScore
0,0.470803,0.480427,0.484099,1.000000,0.556940,0.456000,0.653298,0.851590,0.953778,0.735581,0.756757
1,0.667883,0.594306,0.597173,0.775763,0.972420,0.886222,0.729947,0.674912,0.985778,0.000000,0.540541
2,0.562044,0.533808,0.537102,0.421756,0.741993,0.727111,0.534759,0.491166,0.848000,0.000000,0.979730
3,0.080292,0.056940,0.063604,0.853053,0.395907,0.340444,0.380570,0.174028,0.222222,0.802130,0.907658
4,0.912409,0.879004,0.879859,0.000000,0.160142,0.327111,0.092692,0.058304,0.066667,0.000000,0.172297
...,...,...,...,...,...,...,...,...,...,...,...
172,0.959854,0.320285,0.325088,1.000000,0.733986,0.948444,0.893939,0.884276,0.974222,0.595386,0.593468
173,0.770073,0.743772,0.745583,0.649809,0.287367,0.038222,0.186275,0.172261,0.081778,0.000000,0.171171
174,0.653285,0.633452,0.636042,0.915076,0.540036,0.879111,0.966132,0.695230,0.596444,0.572316,1.000000
175,0.656934,0.647687,0.611307,0.775763,0.787367,0.717333,0.878788,0.565371,0.861333,0.000000,0.408784


In [64]:
normalised_test_df = normalised_test_df.drop(columns ='QScore')
normalised_test_df.head()

,GFN_name,ISO Alpha-2 Code,ISO Alpha-3 Code,crop_land,grazing_land,forest_land,fishing_ground,built_up_land,carbon,total
0,0.470803,0.480427,0.484099,1.000000,0.556940,0.456000,0.653298,0.851590,0.953778,0.735581
1,0.667883,0.594306,0.597173,0.775763,0.972420,0.886222,0.729947,0.674912,0.985778,0.000000
2,0.562044,0.533808,0.537102,0.421756,0.741993,0.727111,0.534759,0.491166,0.848000,0.000000
3,0.080292,0.056940,0.063604,0.853053,0.395907,0.340444,0.380570,0.174028,0.222222,0.802130
4,0.912409,0.879004,0.879859,0.000000,0.160142,0.327111,0.092692,0.058304,0.066667,0.000000


In [65]:
#Logistic Regression
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression()
log_reg.fit(normalised_train_df, y_train_balanced)


LogisticRegression()

In [91]:
from sklearn.metrics import recall_score, accuracy_score, precision_score, f1_score, confusion_matrix
new_predictions = log_reg.predict(normalised_test_df)
new_predictions

array([1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1,
       1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0,
       1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0,
       0])

In [68]:
label_map = {0:'2A',1:'3A'} 
new_pred = pd.Series(new_predictions)
new_pred
y_test_pred_norm  =  new_pred.map(label_map)
print(y_test_pred_norm)
type(y_test_pred_norm)

0      3A
1      3A
2      3A
3      3A
4      3A
       ..
172    3A
173    3A
174    2A
175    2A
176    2A
Length: 177, dtype: object


pandas.core.series.Series

In [69]:
cnf_mat = confusion_matrix(y_true=y_test, y_pred = y_test_pred_norm)
cnf_mat

array([[52, 24],
       [53, 48]], dtype=int64)

In [70]:
accuracy = accuracy_score(y_true=y_test, y_pred=y_test_pred_norm)
print('Accuracy: {}'.format(round(accuracy*100), 2)) #prints 53.0

Accuracy: 56


In [71]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(log_reg, normalised_train_df, y_train_balanced, cv=5, scoring='f1_macro')
scores
#prints	
#array([0.55594592, 0.4733312 , 0.55651249, 0.5245098 , 0.58315241])

array([0.59983994, 0.46952257, 0.41927991, 0.51510204, 0.52525253])

In [72]:
f1 = f1_score(y_true=y_test, y_pred=y_test_pred_norm, pos_label='2A')
print('F1: {}'.format(round(f1*100), 2)) #prints 45.0

F1: 57


In [74]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)
kf.split(normalised_train_df) 
f1_scores = []
#run for every split
for train_index, test_index in kf.split(normalised_train_df):
  x_train, x_test = normalised_train_df.iloc[train_index], normalised_train_df.iloc[test_index]
  y_train, y_test = y_train_balanced[train_index], y_train_balanced[test_index]
  model = LogisticRegression().fit(x_train, y_train)
  #save result to list
  f1_scores.append(f1_score(y_true=y_test, y_pred=model.predict(x_test), 
                   pos_label = 0)*100)
print(f1_scores)

[52.72727272727272, 53.21100917431193, 46.55172413793103, 49.01960784313726, 8.51063829787234]


In [75]:
from sklearn.model_selection import LeaveOneOut
loo = LeaveOneOut()
scores = cross_val_score(LogisticRegression(), normalised_train_df, y_train_balanced, cv=loo, 
                         scoring='f1_macro')
average_score = scores.mean() * 100
average_score

48.99598393574297

In [77]:
y_test.shape

(99,)

In [96]:
 y_test_pred_norm_z = np.resize(new_predictions,99)
 y_test_pred_norm_z   

array([1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1,
       1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0])

In [97]:
precision = precision_score(y_true=y_test, y_pred=y_test_pred_norm_z)
print('Precision: {}'.format(round(precision*100), 2)) #prints 41.0

Precision: 15


In [99]:
recall = recall_score(y_true=y_test, y_pred=y_test_pred_norm_z)
print('Recall: {}'.format(round(recall*100), 2)) #prints 51.0 


Recall: 67


In [106]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
f1_scores = []
#run for every split
for train_index, test_index in skf.split(normalised_train_df, y_train_balanced):
  x_train, x_test = np.array(normalised_train_df)[train_index], np.array(normalised_train_df)[test_index]
  y_train, y_test  = y_train_balanced[train_index], y_train_balanced[test_index]
  model = LogisticRegression().fit(x_train, y_train)
  #save result to list
  f1_scores.append(f1_score(y_true=y_test, y_pred=model.predict(x_test), pos_label=0))
f1_scores

[0.631578947368421,
 0.48484848484848486,
 0.5306122448979592,
 0.543859649122807,
 0.4081632653061225]